# Group 7 Tiny time mixer code. This should work on correctly set up data but I do not have our data so I can not vouch

### TODO Adapt real code from https://github.com/ibm-granite/granite-tsfm/blob/main/notebooks/tutorial/ttm_channel_mix_finetuning.ipynb

### TODO Get real data into Tidy CSV format (ie each row is a column)

### Current code is from https://colab.research.google.com/github/ibm-granite/granite-tsfm/blob/main/notebooks/hfdemo/ttm_getting_started.ipynb

In [ ]:
! conda update -n base -c defaults conda -y
! conda install lxml -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 24.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /users/9/lee02328/.conda/envs/csci8523

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-3.4.0              |       hb9d3cd8_0         2.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.8 MB

The following packages will be UPDATED:

  openssl                                  3.3.2-hb9d3cd8_0 --> 3.4.0-hb9d3cd8_0



openssl-3.4.0        | 2.8 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [ ]:
#! conda install pytorch pytorch-cuda=12.1 -c pytorch -c nvidia -y
#! pip install "tsfm_public[notebooks] @ git+https://github.com/ibm-granite/granite-tsfm.git@v0.2.14" -y

done
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / ^C
failed with repodata from current_repodata.json, will retry with next repodata source.

CondaError: KeyboardInterrupt


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


In [ ]:
import math
import os
import tempfile
import pandas as pd

from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments, set_seed
from transformers.integrations import INTEGRATION_TO_CALLBACK

from tsfm_public import TrackingCallback, count_parameters, load_dataset
from tsfm_public.toolkit.get_model import get_model
from tsfm_public.toolkit.lr_finder import optimal_lr_finder
from tsfm_public.toolkit.visualization import plot_predictions

from tsfm_public import (
    TimeSeriesPreprocessor,
    TrackingCallback,
    count_parameters,
    get_datasets,
)

In [ ]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
#1995
url = "https://apps.glerl.noaa.gov/erddap/griddap/GLSEA_GCS.csvp?sst%5B(1995-01-01T12:00:00Z):1:(2023-12-31T12:00:00Z)%5D%5B(42.05651963):1:(42.05649963)%5D%5B(-87.66870036):1:(-87.66867036)%5D"
data = pd.read_csv(url, on_bad_lines = "warn", delimiter=",", verbose = True)

/tmp/ipykernel_625655/3024688260.py:4: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  data = pd.read_csv(url, on_bad_lines = "warn", delimiter=",", verbose = True)


Tokenization took: 2.06 ms
Type conversion took: 2.78 ms
Parser memory cleanup took: 0.00 ms


In [ ]:
filtered_data = data.dropna()
filtered_data['time (UTC)'] = pd.to_datetime(filtered_data['time (UTC)'])
filtered_data = filtered_data.rename(columns={'time (UTC)': 'date'})
filtered_data = filtered_data.drop(["latitude (degrees_north)", "longitude (degrees_east)"], axis=1)
filtered_data.to_csv("GLSEA_SST_data.csv", index=False)#date
print(filtered_data.head())
_, _, dset_test = load_dataset(
    dataset_name=dataset_name,
    context_length=512,
    forecast_length=1,
    fewshot_fraction=1.0,
    dataset_path="GLSEA_SST_data.csv",
)

                       date  sst (degree_C)
0 1995-01-01 12:00:00+00:00            4.49
1 1995-01-02 12:00:00+00:00            4.43
2 1995-01-03 12:00:00+00:00            4.35
3 1995-01-04 12:00:00+00:00            4.26
4 1995-01-05 12:00:00+00:00            4.17


NameError: name 'dataset_name' is not defined

In [ ]:
# Set seed for reproducibility
SEED = 69
set_seed(SEED)

# TTM Model path. The default model path is Granite-R2. Below, you can choose other TTM releases.
TTM_MODEL_PATH = "ibm-granite/granite-timeseries-ttm-r2"
# TTM_MODEL_PATH = "ibm-granite/granite-timeseries-ttm-r1"
# TTM_MODEL_PATH = "ibm/ttm-research-r2"

# Context length, Or Length of the history.
# Currently supported values are: 512/1024/1536 for Granite-TTM-R2 and Research-Use-TTM-R2, and 512/1024 for Granite-TTM-R1
CONTEXT_LENGTH = 512

# Dataset
# The dataloaders will utilize the easy-to-use YAML configurations defined below.
# Dataset configuration YAMLS: https://github.com/ibm-granite/granite-tsfm/tree/main/tsfm_public/resources/data_config
# Note that `dataset_root_path` can also be provided instead of `dataset_path` to the `load_dataset()` function to
# run this notebook on already downloaded dataset.
# Check the `load_dataset()` function to see more functionalities.
TARGET_DATASET = "etth1"
DATASET_PATH = "GLSEA_SST_data.csv"

# Results dir
OUT_DIR = "ttm_finetuned_models/"

In [47]:
def zeroshot_eval(dataset_name, batch_size, context_length=512, forecast_length=96):
    # Get data
    _, _, dset_test = load_dataset(
        dataset_name=dataset_name,
        context_length=context_length,
        forecast_length=forecast_length,
        fewshot_fraction=1.0,
        dataset_path=DATASET_PATH,
    )

    # Load model
    zeroshot_model = get_model(TTM_MODEL_PATH, context_length=context_length, prediction_length=forecast_length)

    temp_dir = tempfile.mkdtemp()
    # zeroshot_trainer
    zeroshot_trainer = Trainer(
        model=zeroshot_model,
        args=TrainingArguments(
            output_dir=temp_dir,
            per_device_eval_batch_size=batch_size,
            seed=SEED,
            report_to="none",
        ),
    )
    # evaluate = zero-shot performance
    print("+" * 20, "Test MSE zero-shot", "+" * 20)
    zeroshot_output = zeroshot_trainer.evaluate(dset_test)
    print(zeroshot_output)

    # plot
    plot_predictions(
        model=zeroshot_trainer.model,
        dset=dset_test,
        plot_dir=os.path.join(OUT_DIR, dataset_name),
        plot_prefix="test_zeroshot",
        indices=[685, 118, 902, 1984, 894, 967, 304, 57, 265, 1015],
        channel=0,
    )

In [ ]:
def fewshot_finetune_eval(
    dataset_name,
    batch_size,
    learning_rate=None,
    context_length=512,
    forecast_length=96,
    fewshot_percent=5,
    freeze_backbone=True,
    num_epochs=50,
    save_dir=OUT_DIR,
):
    out_dir = os.path.join(save_dir, dataset_name)

    print("-" * 20, f"Running few-shot {fewshot_percent}%", "-" * 20)

    # Data prep: Get dataset
    dset_train, dset_val, dset_test = load_dataset(
        dataset_name,
        context_length,
        forecast_length,
        fewshot_fraction=fewshot_percent / 100,
        dataset_path=DATASET_PATH,
    )

    # change head dropout to 0.7 for ett datasets
    if "ett" in dataset_name:
        finetune_forecast_model = get_model(
            TTM_MODEL_PATH, context_length=context_length, prediction_length=forecast_length, head_dropout=0.7
        )
    else:
        finetune_forecast_model = get_model(
            TTM_MODEL_PATH, context_length=context_length, prediction_length=forecast_length
        )

    if freeze_backbone:
        print(
            "Number of params before freezing backbone",
            count_parameters(finetune_forecast_model),
        )

        # Freeze the backbone of the model
        for param in finetune_forecast_model.backbone.parameters():
            param.requires_grad = False

        # Count params
        print(
            "Number of params after freezing the backbone",
            count_parameters(finetune_forecast_model),
        )

    # Find optimal learning rate
    # Use with caution: Set it manually if the suggested learning rate is not suitable
    if learning_rate is None:
        learning_rate, finetune_forecast_model = optimal_lr_finder(
            finetune_forecast_model,
            dset_train,
            batch_size=batch_size,
        )
        print("OPTIMAL SUGGESTED LEARNING RATE =", learning_rate)

    print(f"Using learning rate = {learning_rate}")
    finetune_forecast_args = TrainingArguments(
        output_dir=os.path.join(out_dir, "output"),
        overwrite_output_dir=True,
        learning_rate=learning_rate,
        num_train_epochs=num_epochs,
        do_eval=True,
        evaluation_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        dataloader_num_workers=8,
        report_to="none",
        save_strategy="epoch",
        logging_strategy="epoch",
        save_total_limit=1,
        logging_dir=os.path.join(out_dir, "logs"),  # Make sure to specify a logging directory
        load_best_model_at_end=True,  # Load the best model when training ends
        metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
        greater_is_better=False,  # For loss
        seed=SEED,
    )

    # Create the early stopping callback
    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=10,  # Number of epochs with no improvement after which to stop
        early_stopping_threshold=1e-5,  # Minimum improvement required to consider as improvement
    )
    tracking_callback = TrackingCallback()

    # Optimizer and scheduler
    optimizer = AdamW(finetune_forecast_model.parameters(), lr=learning_rate)
    scheduler = OneCycleLR(
        optimizer,
        learning_rate,
        epochs=num_epochs,
        steps_per_epoch=math.ceil(len(dset_train) / (batch_size)),
    )

    finetune_forecast_trainer = Trainer(
        model=finetune_forecast_model,
        args=finetune_forecast_args,
        train_dataset=dset_train,
        eval_dataset=dset_val,
        callbacks=[early_stopping_callback, tracking_callback],
        optimizers=(optimizer, scheduler),
    )
    finetune_forecast_trainer.remove_callback(INTEGRATION_TO_CALLBACK["codecarbon"])

    # Fine tune
    finetune_forecast_trainer.train()

    # Evaluation
    print("+" * 20, f"Test MSE after few-shot {fewshot_percent}% fine-tuning", "+" * 20)
    fewshot_output = finetune_forecast_trainer.evaluate(dset_test)
    print(fewshot_output)
    print("+" * 60)

    # plot
    plot_predictions(
        model=finetune_forecast_trainer.model,
        dset=dset_test,
        plot_dir=os.path.join(OUT_DIR, dataset_name),
        plot_prefix="test_fewshot",
        indices=[685, 118, 902, 1984, 894, 967, 304, 57, 265, 1015],
        channel=0,
    )

In [72]:
zeroshot_eval(dataset_name=TARGET_DATASET, context_length=CONTEXT_LENGTH, forecast_length=96, batch_size=64)

INFO:p-625655:t-139937847189824:data_handling.py:load_dataset:Dataset name: etth1, context length: 512, prediction length 96


ValueError: Selection would result in an empty time series, please check start_index and time series length

In [ ]:
fewshot_finetune_eval(
    dataset_name=TARGET_DATASET,
    context_length=CONTEXT_LENGTH,
    forecast_length=96,
    batch_size=64,
    fewshot_percent=5,
    learning_rate=0.001,
)